# Business Intelligence II - Group 2
Created by: Marvin Bathke, Tjorven Beckedorf, Kevin Tiet and Cem Yesil

## Dataset: 2015 Flight Delays and Cancellations
Link to data source: https://www.kaggle.com/datasets/usdot/flight-delays

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium as fl
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
airlines_data = pd.read_csv('airlines.csv')
airports_data = pd.read_csv('airports.csv')
flights_data = pd.read_csv('flights.csv', dtype={"SCHEDULED_DEPARTURE": "string",
                                                       "DEPARTURE_TIME": "string",
                                                       "WHEELS_OFF": "string",
                                                       "WHEELS_ON": "string",
                                                       "SCHEDULED_ARRIVAL": "string",
                                                       "ARRIVAL_TIME": "string"})

FileNotFoundError: [Errno 2] No such file or directory: 'airlines.csv'

## 2. Data Understanding

In [ ]:
airlines_data.head()

In [ ]:
airlines_data.describe()

In [ ]:
airlines_data.isnull().sum()

In [ ]:
#@title
airports_data.head()

In [ ]:
airports_data.describe()

In [ ]:
airports_data.isnull().sum()

In [ ]:
flights_data.head()

In [ ]:
flights_data.describe()

In [ ]:
flights_data.isnull().sum()

### Data Description

Data Description for the flights table:
- **YEAR**: The year the flight arrived.
- **MONTH**: The month the flight arrived.
- **DAY**: The day the flight arrived.
- **DAY_OF_WEEK**: The weekday the flight arrived encoded in int from 1 to 7.
- **AIRLINE**: The IATA_CODE of the airline of the flight.
- **FLIGHT_NUMBER**: Number of the flight.
- **TAIL_NUMBER**: An identification number for airplanes. 
- **ORIGIN_AIRPORT**: The IATA_CODE of the airport the airplane is scheduled to depart.
- **DESTINATION_AIRPORT**: The IATA_CODE of the airport the airplane is scheduled to land.
- **SCHEDULED_DEPARTURE**: Scheduled departure time encoded as HHMM String
- **DEPARTURE_TIME**: The time it takes from the closing of the gate to the actual wheels off. Also encoded as HHMM String.
- **DEPARTURE_DELAY**: The delay of the departure in min between the real and the planned departure.
- **TAXI_OUT**: The time duration elapsed between departure from the origin airport gate and wheels off.
- **WHEELS_OFF**: The actual time the airplane departed as HHMM String.
- **SCHEDULED_TIME**: The planned time the flight trip is needed.
- **ELAPSED_TIME**: The form TAXI_IN to TAXI_OUT.
- **AIR_TIME**: The duration between wheels_on and wheels_off.
- **DISTANCE**: Distance between the two airports.
- **WHEELS_ON**: The time point that the aircraft's wheels touch on the ground.
- **TAXI_IN**: The time duration elapsed between wheels-on and gate arrival at the destination airport.
- **SCHEDULED_ARRIVAL**: Planned arrival time as HHMM String.
- **ARRIVAL_TIME**: The time of WHEELS_IN + TAXI_IN
- **ARRIVAL_DELAY**: The delay of the arrival in min between the real and the scheduled arrival.
- **DIVERTED**: Aircraft got diverted to another airport. 0 = no; 1 = yes
- **CANCELLED**: Flight got cancelled. 0 = no ; 1 = yes.
- **CANCELLATION_REASON**: Reason why the flight is cancelled. A= Airline/Carrier, B= Weather, C= National Air System, D = Security
- **AIR_SYSTEM_DELAY**: Delay caused by air system in minutes
- **SECURITY_DELAY**: Delay caused by security in minutes
- **AIRLINE_DELAY**:  Delay caused by the airline in minutes
- **LATE_AIRCRAFT_DELAY**: Delay caused by aircraft in minutes.
- **WEATHER_DELAY**: Delay caused by weather in minutes.

In [ ]:
flights_data.head()

## 3. Data Preprocessing

### 3.1 Cleaning the data

Convert all columns to lower case

In [ ]:
flights_data.columns = map(str.lower, flights_data.columns)
airlines_data.columns = map(str.lower, airlines_data.columns)
airports_data.columns = map(str.lower, airports_data.columns)

Firstly, make the date columns to an actual date. Using datetime and a dateformat is more convenient to work with.

In [ ]:
flights_data["date"] = pd.to_datetime(flights_data[["year", "month", "day"]])

After that we handle the HHMM Strings and format them to also a datetime-format. 

In [ ]:
def format_timestamp(to_convert):
    if pd.isna(to_convert):
      return pd.NA
    else:
      if to_convert == "2400":
        to_convert = "0000"
        return datetime.time(int(to_convert[0:2]), int(to_convert[2:4]))
      return datetime.time(int(to_convert[0:2]), int(to_convert[2:4]))
      #to_convert = pd.to_datetime(to_convert.zfill(4), errors="coerce", format="%H%M")


In [ ]:
flights_data["scheduled_departure"] = flights_data["scheduled_departure"].apply(format_timestamp)
flights_data["departure_time"] = flights_data["departure_time"].apply(format_timestamp)
flights_data["wheels_off"] = flights_data["wheels_off"].apply(format_timestamp)
flights_data["wheels_on"] = flights_data["wheels_on"].apply(format_timestamp)
flights_data["scheduled_arrival"] = flights_data["scheduled_arrival"].apply(format_timestamp)
flights_data["arrival_time"] = flights_data["arrival_time"].apply(format_timestamp)

Next we can add more columns to use later for the analysis.
The `delay_difference` is the difference between the departure and arrival delay.
The `cat`-suffix maps attributes with non-numeric types to numbers.

In [ ]:
flights_data["delay_difference"] = flights_data.arrival_delay - flights_data.departure_delay
flights_data['airline_cat']=flights_data['airline'].astype('category').cat.codes
flights_data['origin_airport_cat']=flights_data['origin_airport'].astype('category').cat.codes
flights_data['destination_airport_cat']=flights_data['destination_airport'].astype('category').cat.codes

# 4. Exploratory Data Analysis
Exploratory Data Analysis (EDA) aims to make Data and its inherent patterns understandable for humans (by making pretty plots).
We begin by looking at generally at all the Data. Afterwards we focus on airports and flights.

The EDA of this project consists of three parts: A general EDA, an EDA in relationship with the airline data and a third one in relation to the airports' data.
The questions /hypotheses which were created for and during the EDA are:
- Where are the airports located on a map?
- Which airport has the most departures / arrivals? Is one of the most visited airports a one-way airport?
- Which airport has the most numbers delays? 
- Which airport has the most time of delay?
- Does the number of flights correlate with the delay time? Is a high number of flights causing delays?
- Which airlines have the highest delays?
- What type of delays cause the most delay?
- Can we predict a flights destination based on its attributes?
- Can we predict the delay time based on its attributes?z

### 4.1 General EDA
First we take a look at some general metrics.

In [ ]:
to_describe = ["departure_delay", "arrival_delay", "scheduled_time", "elapsed_time", "air_time", "delay_difference", "distance", 'cancelled', 'cancellation_reason',
               'air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']
flights_data[to_describe].describe().applymap(lambda x: f"{x:.3f}") # describe data with pretty print

todo insights to describe


Next we can look at histograms made from the attributes of `flight_data`.


In [ ]:
flights_data.hist(figsize=(20,20))

January and December have significantly more flights than the rest of the months.
The last day of the month has more flight than the other days.
There are fewer flights on Saturday.
The duration of most flights is in between of 100 and 200 minutes.
The duration of the air time of most flights is inbetween 0 and 166 minutes. Taking the general metrics account the interval is between 7 and 166 minutes.
The travel distance of most flights is in between 0 and 1000 Miles. Taking the general metrics account the interval is between 21 and 1000 Miles.

As next step we can look at the correlation between the attributes of `flight_data`. In order to achieve this, we use the pearson correlation and visualize the matrix as heatmap.

In [ ]:
continuous_of_interest = ["date", "day_of_week", "distance","scheduled_time","elapsed_time", "scheduled_departure", "departure_delay","arrival_delay", "delay_difference","airline_cat", "origin_airport_cat", "destination_airport_cat"] # airport, airline
f_corr, ax_corr = plt.subplots(figsize=(20, 12))
corr = flights_data.loc[:, continuous_of_interest].corr(method="pearson")
hm_corr = sns.heatmap(round(corr,2), annot=True, ax=ax_corr, cmap="coolwarm",fmt='.2f', linewidths=.05)
f_corr.subplots_adjust(top=0.93)
t= f_corr.suptitle('Attribute Pearson Correlation Heatmap', fontsize=14)


*   `distance`, `scheduled_time` and `elapsed_time` are all correlated with each other
*   `departure_delay` and `arrival_delay` are correlated
*   `origin_airport_cat` and `destination_airport_cat` are correlated
*   `airline_cat` is negatively correlated with `distance`, `scheduled_time` and `elapsed_time`

### 4.2. Analysis of the Airport delays
The general idea about this part of EDA is it to get an insight about delays in regard to the airports.

##### In order to get a first impression of the distribution how many flights are departed from each airport, I group and count them.

In [ ]:
flights_data.groupby(['origin_airport']).count()

##### Investigating flights with an unusual `origin_airport`

We noticed that there are flights where the `origin_airport` is titled with a number instead of a three-lettered-code as usual.

In [ ]:
number_of_flights_with_number_as_origin_airport = 0
for i in range(len(flights_data.origin_airport)):
  origin_airport_len = len(flights_data.origin_airport.loc[i]) if type(flights_data.origin_airport.loc[i]) == str else flights_data.origin_airport.loc[i]
  if origin_airport_len > 3:
    number_of_flights_with_number_as_origin_airport+=1
print(number_of_flights_with_number_as_origin_airport)

In [ ]:
print("Amount of flights where the origin airport is 10135:", len(flights_data[flights_data.origin_airport == 10135]))
print("Amount of flights where the origin airport is 10136:", len(flights_data[flights_data.origin_airport == 10136]))
print("Amount of flights where the origin airport is 10140:", len(flights_data[flights_data.origin_airport == 10140]))

In [ ]:
flights_data[flights_data.origin_airport == 10140]

When briefly looking into the flights, we noticed that all flights which have a numerical origin_airport take place in october.

In [ ]:
flights_data[flights_data.month == 10]

What we find out is that the amount of flights with a numerical `origin_airport` is equal to the amount of flights in October. We assume that this is a data collection error or systematic error in the system where the data is collected.

At the same time there seems to be no real hint how a numerical origin_airport value can be mapped to an `IATA_CODE`.

To deal with it, we are merging the `airports_data` and `flights_data`. This should remove the data, which can't be mapped to a "properly" collected airport.

At the same time we can gain more readable information. We can't just see the three-lettered-code of an airport, we also look up its name.

##### Merging flight data with airports data:

In [ ]:
airports_flight_data = flights_data.merge(airports_data, left_on="origin_airport", right_on="iata_code")
airports_flight_data = airports_flight_data.rename(columns={"airport": "origin_airport_name"})
airports_flight_data = airports_flight_data.drop(columns = ["iata_code"])
airports_flight_data.head()

Next, we drop the attributes which aren't necessary for the analysis.

In [ ]:
airports_flight_data_orig = airports_flight_data.copy()

In [ ]:
airports_flight_data = airports_flight_data.drop(columns = ['airline', 'flight_number', 'tail_number', 'elapsed_time', 'wheels_off', 'wheels_on', 'distance', 'taxi_in', 'taxi_out', 'day_of_week', 'air_time', 'country', 'state'])

##### We want to look at the question of which airports are the biggest in terms of the number of in-and-out-going traffic / flights:

In [ ]:
number_of_flights_per_origin_airport = airports_flight_data.groupby(by='origin_airport').size()
number_of_flights_per_origin_airport = number_of_flights_per_origin_airport.sort_values(ascending=False)

number_of_flights_per_destination_airport = airports_flight_data.groupby(by='destination_airport').size()
number_of_flights_per_destination_airport = number_of_flights_per_destination_airport.sort_values(ascending=False)

In [ ]:
print('Top 10 destination airports:', number_of_flights_per_destination_airport.index[:10])
print('Top 10 origin airports:', number_of_flights_per_origin_airport.index[:10])

The order of the top 10 airports is identical in comparison of `destination_airport` and `origin_airport`.

In [ ]:
f, axes = plt.subplots(figsize = (20,15))
X = number_of_flights_per_origin_airport.index[:10]
y_origin = number_of_flights_per_origin_airport.values[:10]
y_dest = number_of_flights_per_destination_airport.values[:10]
  
X_axis = np.arange(10)
  
plt.bar(X_axis - 0.2, y_origin, 0.4, label = 'Origin')
plt.bar(X_axis + 0.2, y_dest, 0.4, label = 'Destination')
  
plt.xticks(X_axis, X)
plt.xlabel("Airport IATA Code")
plt.ylabel("Number of flights")
plt.title('Top 10 Airports with the most flights (without October)', fontsize=15)
plt.legend()
plt.show()

In [ ]:
airports_data[airports_data['iata_code'] == 'ATL']

In [ ]:
airports_data[airports_data['iata_code'] == 'ORD']

In [ ]:
airports_data[airports_data['iata_code'] == 'DFW']

The bar charts show the Top 10 Airports (Origin and Destination) with the most flights out of this dataset. The month October was excluded from this visualization due to the problem that it is different encoded. In both charts can be seen that the order of origin and destination airports are the same (sorted by the amount of flights). 

The top three airports with the most in-and-outgoing flight traffic are:
- Hartsfield-Jackson Atlanta International
- Chicago O'Hare International Airport
- Dallas/Fort Worth International Airport

##### Next, we want to investigate which airports have the most delays:
To investigate this, we want to look at:
1. the time of the flight delays
2.  the number of flights with delays


##### First investigation of the delay values:
First of what can be noticed is that there are different kinds of delays:
- `departure_delay`
- `arrival_delay`
- `air_system_delay`
- `security_delay`
- `airline_delay`
- `late_aircraft_delay`
- `weather_delay`

At first glance on the NaN values at the beginning, we already saw that nearly every delay attribute had `NaN` values. Therefore, we first need to understand when those values occur.

In [ ]:
airports_flight_data[airports_flight_data['cancelled'] == 0].isnull().sum()

In [ ]:
print('Number of cancelled flights:', len(airports_flight_data[airports_flight_data['cancelled'] == 1]))
airports_flight_data[airports_flight_data['cancelled'] == 1].isnull().sum()

The first attribute (`departure_delay`) does never have a `NaN` value when a flight is not cancelled. When looking at the other attributes, they do have NaN values. Especially the attributes `air_system_delay`, `security_delay`, `airline_delay`, `late_aircraft_delay` and `weather_delay` have a huge amount of `NaN` value for not-cancelled flights. At first, an assumption was that delays of these types lead to an increased probability of cancellation. But when investigating the cancelled flights, every entry was a `NaN` value for those attributes.

##### Next we want to look at the different delay attributes and how they are related to each other:

Therefore, we looked at a row where the columns `air_system_delay`, `security_delay`, `airline_delay`, `late_aircraft_delay` and `weather_delay` are not `NaN`:

In [ ]:
airports_flight_data.loc[25:35,['origin_airport','departure_delay', 'arrival_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']]

The following conclusion is only based on a brief look at the data.
Our presumption is that the attributes `air_system_delay`, `security_delay`, `airline_delay`, `late_aircraft_delay` and `weather_delay` are a more detailed representation of the attribute `arrival_delay`.

We chose to look only in the columns `air_system_delay`, `security_delay`, `airline_delay`, `late_aircraft_delay` and `weather_delay` due to the fact that the `arrival_delay` represents the net loss of time on a flight. Furthermore, we can get a more detailed view when looking at those attributes instead of the `departure_delay`.

As previously stated that the `arrival_delay` is only a summarization of the more detailed attributes, we can also leave this out for the next steps.

#### 1. Investigation of the time of the flight delays per airport

For the calculation of delay time it is necessary to fill the `NaN` values of the attributes `air_system_delay`, `security_delay`, `airline_delay`, `late_aircraft_delay` and `weather_delay`. Otherwise mathematical operations like building the sum wouldn't be possible.

##### NaN-fill of the airport_flight data:

This means that those values need to be transformed in order to use them for the second part of the question. We chose to fill those `NaN` values with 0.

In [ ]:
airports_flight_data_na_filled_orig = airports_flight_data_orig.copy()
airports_flight_data_na_filled_orig[['air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']] = airports_flight_data_na_filled_orig[['air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']].fillna(value=0)
airports_flight_data_na_filled_orig.loc[:,['air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']].isnull().sum()

In [ ]:
airports_flight_data_na_filled = airports_flight_data.copy()
airports_flight_data_na_filled[['air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']] = airports_flight_data_na_filled[['air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']].fillna(value=0)
airports_flight_data_na_filled.loc[:,['air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']].isnull().sum()

##### Investigating negative `arrival_delay`:
We also noticed that a big amount of flights show a negative value for `arrival_delay`. In this part of the analysis we want only focus on the "real" delay time. Otherwise positive and negative values could cancel each other out.

In [ ]:
len(airports_flight_data_na_filled[airports_flight_data_na_filled['arrival_delay'] < 0])

In total there are more than 3 million data entries having a negative `arrival_delay`. Therefore, the impact when not handling it, could be huge.

In [ ]:
airports_flight_data_na_filled.iloc[0][['arrival_delay', 'security_delay', 'air_system_delay', 'airline_delay', 'weather_delay']]

Fortunately, we found out that the detailed attributes are all set to zero if a flight is earlier than expected. That means, handling it is not necessary.

##### Creating the summed delays grouped by origin airport:

In [ ]:
flight_delays_grouped_by_origin_airport_in_minutes = airports_flight_data_na_filled.loc[:,['origin_airport','air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']].groupby(by='origin_airport').sum()
flight_delays_grouped_by_origin_airport_in_minutes.head()

It is already visible that the security delay and weather delay seem to be smaller than the other types of delay.

In [ ]:
airports_flights_delay_types = pd.DataFrame((airports_flight_data_na_filled.groupby(["origin_airport"])["air_system_delay"].sum()/60).round(2))
airports_flights_delay_types["security_delay"] = pd.DataFrame((airports_flight_data_na_filled.groupby(["origin_airport"])["security_delay"].sum()/60).round(2))
airports_flights_delay_types["airline_delay"] = pd.DataFrame((airports_flight_data_na_filled.groupby(["origin_airport"])["airline_delay"].sum()/60).round(2))
airports_flights_delay_types["late_aircraft_delay"] = pd.DataFrame((airports_flight_data_na_filled.groupby(["origin_airport"])["late_aircraft_delay"].sum()/60).round(2))
airports_flights_delay_types["weather_delay"] = pd.DataFrame((airports_flight_data_na_filled.groupby(["origin_airport"])["weather_delay"].sum()/60).round(2))
airports_flights_delay_types["sum"] = airports_flights_delay_types[["air_system_delay", "security_delay","airline_delay", "late_aircraft_delay", "weather_delay"]].sum(axis=1)
airports_flights_delay_types = airports_flights_delay_types.sort_values("sum", ascending=False)

In [ ]:
temp = pd.DataFrame()
airports_flights_delay_sum = airports_flights_delay_types["sum"]
temp['security_delay_per'] = pd.DataFrame(((airports_flights_delay_types["security_delay"] /airports_flights_delay_sum)*100).round(2))
temp["air_system_delay_per"] = pd.DataFrame(((airports_flights_delay_types["air_system_delay"] /airports_flights_delay_sum)*100).round(2))
temp["airline_delay_per"] = pd.DataFrame(((airports_flights_delay_types["airline_delay"] /airports_flights_delay_sum)*100).round(2))
temp["late_aircraft_delay_per"] = pd.DataFrame(((airports_flights_delay_types["late_aircraft_delay"] /airports_flights_delay_sum)*100).round(2))
temp["weather_delay_per"] = pd.DataFrame(((airports_flights_delay_types["weather_delay"] /airports_flights_delay_sum)*100).round(2))

In [ ]:
temp

In [ ]:
f, ax = plt.subplots(1, 2, figsize = (20,10))
ax[0].set(ylabel="Airport IATA Code",
       xlabel="Flight delay time in hours")

ax[0].set_title('Top 10 Origin Airports with most delay time in hours')

ax[1].set(ylim=(-0.5, 10.5), ylabel="Airport IATA Code",
       xlabel="Flight delay time percentage")
ax[1].set_title('Top 10 Origin Airports with most delay time: percentual distribution')

ax[0] = airports_flights_delay_types[['air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']].iloc[:10].plot.barh(stacked=True, ax=ax[0])
ax[1] = temp[['air_system_delay_per', 'security_delay_per', 'airline_delay_per', 'late_aircraft_delay_per', 'weather_delay_per']].iloc[:10].plot(kind="barh",stacked=True, ax=ax[1])
plt.suptitle('Top 10 origin airports with the most delay time', fontsize= 15)
plt.show()

In every airport of the plot a big amount of delay time is caused by `airline_delay` and `late_aircraft_delay`. In contrast to that the `security_delay` time is just minimal. 

Comparing this rank order of the delay time to the plot with the largest amount of in- and outgoing flights, the ranking is about the same. In this plot (Top 10 origin airports with the most delay time), the airports `LGA` (LaGuardia Airport (Marine Air Terminal)) and `MCO` (Orlando International Airport) are ranked 9th and 10th, respectively. In contrast to the top 10 airports with the most flights, `PHX` (Phoenix Sky Harbor International Airport) and `MSP` (Minneapolis-Saint Paul International) are missing here.

In addition, the rank order did change a little. Here is a table showing the Top-10 rank order of both plots:

| Rank | Top 10 with most delay time | Top 10 with most flights |
|------|-----------------------------|--------------------------|
| 1    | ORD                         | ATL                      |
| 2    | ATL                         | ORD                      |
| 3    | DFW                         | DFW                      |
| 4    | DEN                         | DEN                      |
| 5    | LAX                         | LAX                      |
| 6    | IAH                         | SFO                      |
| 7    | SFO                         | PHX                      |
| 8    | LAS                         | IAH                      |
| 9    | LGA                         | LAS                      |
| 10   | MCO                         | MSP                      |

The similarity of these two diagrams can be explained by the fact that a higher number of flights also results in a higher total delay time.

##### Next, we want to look at the ratio between the delay time and number of flights. In this way, we can see what the average delay per flight is at an airport and whether it is similar to the number of flights plot.

In [ ]:
airports_delay_num_flights_df = airports_flights_delay_types.merge(number_of_flights_per_origin_airport.rename('number_of_flights'), left_index=True, right_index=True)
airports_delay_num_flights_df.head()

In [ ]:
temp2 = pd.DataFrame()
number_of_flights = airports_delay_num_flights_df['number_of_flights']
temp2['security_delay_ratio'] = pd.DataFrame(((airports_delay_num_flights_df["security_delay"] /number_of_flights).round(2)))
temp2["air_system_delay_ratio"] = pd.DataFrame(((airports_delay_num_flights_df["air_system_delay"] /number_of_flights).round(2)))
temp2["airline_delay_ratio"] = pd.DataFrame(((airports_delay_num_flights_df["airline_delay"] /number_of_flights).round(2)))
temp2["late_aircraft_delay_ratio"] = pd.DataFrame(((airports_delay_num_flights_df["late_aircraft_delay"] /number_of_flights).round(2)))
temp2["weather_delay_ratio"] = pd.DataFrame(((airports_delay_num_flights_df["weather_delay"] /number_of_flights).round(2)))
temp2["sum_delay_ratio"] = pd.DataFrame(((airports_delay_num_flights_df["sum"] /number_of_flights).round(2)))
temp2 = temp2.sort_values('sum_delay_ratio', ascending=False)

In [ ]:
f, ax = plt.subplots(figsize = (10,10))
ax.set(ylabel="Airport IATA Code",
       xlabel="Mean flight delay time in minutes")

temp2[['air_system_delay_ratio', 'security_delay_ratio', 'airline_delay_ratio', 'late_aircraft_delay_ratio', 'weather_delay_ratio']].iloc[:10].plot.barh(stacked=True, ax=ax)
plt.title('Top 10 origin airports with the highest delay time ratio', fontsize= 15)
plt.show()

In [ ]:
temp2.index[:10]

In [ ]:
highest_delay_ratio_airports_ranking_in_num_of_flights = airports_flight_data_na_filled.groupby('origin_airport').size()
highest_delay_ratio_airports_ranking_in_num_of_flights = highest_delay_ratio_airports_ranking_in_num_of_flights.sort_values(ascending=False)
highest_delay_ratio_airports_ranking_in_num_of_flights = highest_delay_ratio_airports_ranking_in_num_of_flights.reset_index()
highest_delay_ratio_airports_ranking_in_num_of_flights = highest_delay_ratio_airports_ranking_in_num_of_flights[highest_delay_ratio_airports_ranking_in_num_of_flights.origin_airport.isin(temp2.index[:10])]
highest_delay_ratio_airports_ranking_in_num_of_flights

Summed Number of flights of the top 10 airports with the highest delay ratio:

In [ ]:
highest_delay_ratio_airports_ranking_in_num_of_flights[0].sum()

In [ ]:
airports_flight_data_na_filled.groupby('origin_airport').size().sort_values(ascending=False)

In [ ]:
temp2_ri = temp2.reset_index()

In [ ]:
temp2_ri[temp2_ri['origin_airport'].isin(number_of_flights_per_origin_airport.index[:10])]

The plot and table shows that none of airports out of the _Top 10 origin airports with the largest number of flights_ or _Top 10 origin airports with the most delay time_ are part of the _Top 10 origin airports with the highest delay time ratio_.

Instead, the Top 3 airports of both the number of flights and as well the plot with the most delay time in total are ranked at:

| origin_airport | Index of the delay time ratio DF |
|----------------|----------------------------------|
| ATL            | 256                              |
| ORD            | 41                               |
| DFW            | 117                              |

It was shown that the ratio of the delay time per flight seems not to be strongly connected to the number of flights. But it needs to be considered that the airports with the highest delay ratio also do not have many registered flights. The summed number of flights of the top 10 airports with the highest delay ratio is 6706. Comparing this number to only the number of flights of ATL with 346.836, these 10 airports just registered about 1,9% of the flights.

The conclusion that the number of flights and the delay ratio are not strongly connected is still valid when considering the index ranking,e.g. ATL 256 out of 322. Nevertheless, there are airports which have just a small amount of flights and are not a good representation. That is the reason to take this conclusion with caution. To back up this conclusion we want to look at the correlation between those variables in the next step.

##### Looking at the correlation between the avg delay time per flight and number of flights

In [ ]:
correation_df_number_of_flights_avg_delay = temp2.merge(number_of_flights.rename('number_of_flights'), left_index=True, right_index=True)
correation_df_number_of_flights_avg_delay = correation_df_number_of_flights_avg_delay.sort_values('number_of_flights', ascending=False)
correation_df_number_of_flights_avg_delay.head()

In [ ]:
sns.heatmap(correation_df_number_of_flights_avg_delay[['sum_delay_ratio', 'number_of_flights']].corr(), cmap="YlGnBu", annot=True)

In [ ]:
sns.regplot(data=correation_df_number_of_flights_avg_delay, x='sum_delay_ratio', y='number_of_flights',
                line_kws={"color": "green"},
                order=2)

The correlation matrix and scatter plot clearly shows that the correlation of these two variables is quite weak.

Reflecting on the two initial questions:
- Does the number of flights correlate with the delay time? 
- Is a high number of flights causing delays?

There seems to be a correlation between number of flights and delay time. But when considering only the average delay time per flight, there is no pattern.
That means that a high number of flights does not cause a delay.


#### 2. Investigation of the number of flights with delays per airport

In [ ]:
number_of_flight_delays_grouped_by_origin_airport = airports_flight_data.loc[:,['origin_airport','security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay']].groupby(by='origin_airport').apply(lambda x: x.count())
number_of_flight_delays_grouped_by_origin_airport = number_of_flight_delays_grouped_by_origin_airport.sort_values(ascending=False, by='security_delay')
number_of_flight_delays_grouped_by_origin_airport

When counting the number of the number of delays of each attribute, we have noticed that whenever one of those attributes has a delay, every other of these attributes has a value. If one of these hasn't a delay, its value is set to `0.0`. 

Due to the fact that the number of delays for each of these attributes is going to be the same, we are only plotting one of the attributes to represent the other delay attributes as well. We chose the attribute `security_delay`.

In [ ]:
#@title
f, ax = plt.subplots(figsize = (15,15))
sns.barplot(y=number_of_flight_delays_grouped_by_origin_airport.index, x=number_of_flight_delays_grouped_by_origin_airport.security_delay)
ax.set(ylim=(-0.5, 10.5), ylabel="Airport IATA Code",
       xlabel="Number of flights with delay")
#ax.set_xticklabels(number_of_flights_per_origin_airport.index)
for p in ax.patches:
  ax.annotate(int(p.get_width()), (p.get_width()-5000, p.get_y()+0.35), fontsize=15)

plt.title('Top 10 Origin Airports with the highest number of flight delays due to security / airline / late aircraft or weather')
plt.show()

In [ ]:
security_flight_delays_grouped_by_origin_airport_in_minutes = flight_delays_grouped_by_origin_airport_in_minutes['security_delay'].sort_values(ascending=False)
airline_flight_delays_grouped_by_origin_airport_in_minutes = flight_delays_grouped_by_origin_airport_in_minutes['airline_delay'].sort_values(ascending=False)
late_aircraft_flight_delays_grouped_by_origin_airport_in_minutes = flight_delays_grouped_by_origin_airport_in_minutes['late_aircraft_delay'].sort_values(ascending=False)
weather_flight_delays_grouped_by_origin_airport_in_minutes = flight_delays_grouped_by_origin_airport_in_minutes['weather_delay'].sort_values(ascending=False)

#### 4.2.1 Geographical Airport Distribution
1. Where are the airports located on the map?


In [ ]:
airportsMapData = airports_data
airportsMapData

In [ ]:

map = fl.Map(location=[40, -98], zoom_start=5)
airportsMapData.dropna(inplace=True)

In [ ]:
for airport in range(0, len(airportsMapData)):
    fl.Marker([airportsMapData.iloc[airport]["latitude"], airportsMapData.iloc[airport]["longitude"]], popup=airportsMapData.iloc[airport]["iata_code"], tooltip=airportsMapData.iloc[airport]["airport"]).add_to(map)

map

### 4.3 Airline Flight delays


In [ ]:
flights_data = flights_data.merge(airlines_data, left_on="airline", right_on="iata_code")
flights_data = flights_data.rename(columns={"airline_x": "airline_code", "airline_y": "airline_name"})
flights_data = flights_data.drop(columns = ["iata_code"])

Since we want to analyze the delay, we drop every table that we don't need for the analysis. For this we focus on the actual departure times and not on the taxi and wheel times, since the departure time is that time a passenger cares about. We also drop the cancelled flights, since there are many null values for the delay (because the flight never departed).

In [ ]:
flights_data_delay = flights_data.drop(columns= ["day_of_week", "flight_number", "tail_number", 
                            "taxi_out", "wheels_off",'air_time', 'distance',
                            "wheels_on", "taxi_in"])
flights_data_delay = flights_data_delay[flights_data_delay["cancelled"] ==0]
flights_data_delay

In the next steps the describe function is used to plot some statistical values. At this point the `depature_delay` is used, since you can argue that the airline can influence the `arrival_delay` by e.g. adjusting the flight speed. This also explains a difference in these both values and why the `depature_delay` is used in the analysis.

In [ ]:
statistical_values_delay = flights_data_delay["departure_delay"].groupby(flights_data["airline_name"]).describe()
statistical_values_delay

First lets look at the number of flight by airlines .

In [ ]:
sns.barplot(y=statistical_values_delay.index, x ="count", data = statistical_values_delay, order =statistical_values_delay.sort_values("count").index)

We can look at the mean of the delay to evaluate which airline has the most delays.

In [ ]:
sns.barplot(y=statistical_values_delay.index, x ="mean", data = statistical_values_delay, order =statistical_values_delay.sort_values("mean").index)

The plot above shows the Airlines with the avg delay in min. The problem here now is, that if you look at the max values, there are some delays that are over a day long. Since this are still delays, we could split the delays into certain ranges. Since the avg delay is ~ 9min I would determine that as a short delay. After that you could argue that a delay up to an hour is a medium and over an hour could be described as a long delay.

In [ ]:
rangelist = [10,60]
flights_data_delay["delay_range"] =  pd.cut(flights_data_delay['departure_delay'], 
                              [-np.inf] + sorted(rangelist) + [np.inf])

In [ ]:
delay_ranges_airlines = flights_data_delay[["delay_range", "airline_name"]]

In [ ]:
delay_range_count = delay_ranges_airlines.groupby(['delay_range', 'airline_name']).size().unstack(fill_value=0)
delay_range_count = delay_range_count.T
delay_range_count = delay_range_count.rename(columns={delay_range_count.columns[0]: "<10", delay_range_count.columns[1]: "10-60", delay_range_count.columns[2]: ">60"})
delay_range_count.plot(kind="bar", title="Delay displayed in ranges (in minutes)",stacked=True)


In [ ]:
delay_range_count["sum_delays"] = delay_range_count[["<10","10-60", ">60"]].sum(axis=1)
delay_range_count["<10_in_per"] = ((delay_range_count["<10"] /delay_range_count["sum_delays"])*100).round(2)
delay_range_count["10-60_in_per"] = ((delay_range_count["10-60"] /delay_range_count["sum_delays"])*100).round(2)
delay_range_count[">60_in_per"] = ((delay_range_count[">60"] /delay_range_count["sum_delays"])*100).round(2)

delay_range_count_percent = delay_range_count.drop(columns=["<10","10-60", ">60", "sum_delays"])
delay_range_count_percent = delay_range_count_percent.sort_values("<10_in_per")
delay_range_count_percent.plot(kind="bar", stacked=True)

This gives us some insights why the airlines haves the highest means. Spirit Airlines have ~10% of the flights with a delay of more than an hour and by this have the highest amount of long delays. On second place with the highest amount of long delays is Frontier Airlines Inc., shortly followed by United Airlines Inc and JetBlue Airways. One could argue that Shortwest Airlines Co. are not in the top five worst Airlines this time, since they manage to have only ~6% long delays even though they have the highest flight volume. Also, the comparison of some Airlines have to be viewed carefully, since the flight volume of e.g. Hawaiian Airlines Inc is ~90% less than the flight volume of Spirit Airlines.

In the next step, the different type of delays get evaluated. To get an insight about we count the numbers of delay and

In [ ]:
flights_delay_types = pd.DataFrame((flights_data_delay.groupby(["airline_name"])["air_system_delay"].sum()/60).round(2))
flights_delay_types["security_delay"] = pd.DataFrame((flights_data_delay.groupby(["airline_name"])["security_delay"].sum()/60).round(2))
flights_delay_types["airline_delay"] = pd.DataFrame((flights_data_delay.groupby(["airline_name"])["airline_delay"].sum()/60).round(2))
flights_delay_types["late_aircraft_delay"] = pd.DataFrame((flights_data_delay.groupby(["airline_name"])["late_aircraft_delay"].sum()/60).round(2))
flights_delay_types["weather_delay"] = pd.DataFrame((flights_data_delay.groupby(["airline_name"])["weather_delay"].sum()/60).round(2))
flights_delay_types = flights_delay_types.sort_values("airline_delay")

In [ ]:
flights_delay_types.plot(kind="barh", stacked=True, figsize=(7,5.5))

As you can see the most time of delays comes form the  late_aircraft_delay. Still you could argue that an aircraft can be late, because of every other type of delay, e.g. an aircraft is grounded because of the weather or a security reason and is scheduled to flight again shortly after landing, it would be late due to another issue. By that conclusion the reason for the most time of delay is because of the airline, shortly followed by air_system_delays. Surprisingly, the delays because of the weather are quite rare in comparison to the just mentioned delays. Security reasons seem to cause the least time for delay and are pretty rare. 

In [ ]:
flights_delay_types["sum"] = flights_delay_types[["air_system_delay", "security_delay","airline_delay", "late_aircraft_delay", "weather_delay"]].sum(axis=1)

In [ ]:
temp = pd.DataFrame()
temp = pd.DataFrame(((flights_delay_types["security_delay"] /flights_delay_types["sum"])*100).round(2))
temp["air_system_delay_pre"] = pd.DataFrame(((flights_delay_types["air_system_delay"] /flights_delay_types["sum"])*100).round(2))
temp["airline_delay_pre"] = pd.DataFrame(((flights_delay_types["airline_delay"] /flights_delay_types["sum"])*100).round(2))
temp["late_aircraft_delay_pre"] = pd.DataFrame(((flights_delay_types["late_aircraft_delay"] /flights_delay_types["sum"])*100).round(2))
temp["weather_delay_pre"] = pd.DataFrame(((flights_delay_types["weather_delay"] /flights_delay_types["sum"])*100).round(2))
temp = temp.rename(columns={temp.columns[0]: "security_delay_pre"})
temp.plot(kind="barh",stacked=True)

As you can see, the plot represents that what was described in the previous cell, but this time its put in percentages values.  

### Conclusion about delays in relation to airlines

The general idea about this part of EDA was to get an insight about delays in regard to the airlines. First, the `airline_name` gets added to the `flights_data`, because the name of the airline makes much more sense for displaying purposes. In the next step, columns that are not needed for the analysis are dropped and the `flights_data` get grouped by the `airline_name` to aggregate data based on the different airlines. 

The first thing displayed is the volume of the airlines. As shown in the plot, there is a huge difference in the volume between airlines. Still even *Virgin America* with the lowest value has more than ~60.000 flights on record, but in comparison to the flight volume over ~1.2 million from *Southwest Airlines Co.* it is small. This could be a factor in comparing the airlines with each other.

In the next plot mean values of the `depature_delay` of the airlines are shown. The `depature_delay` is used and not the `arrival_delay` because the airline can influence the `arrival_delay` by e.g. increasing the flying speed or similar. Since we want to compare values, the `departure_delay` is used, since it is much harder for an airline to influence that. Based on this the airlines with the most delay are *Spirit Air Lines* with a mean value of ~16 minutes, *United Air Lines Inc.* with a little over 14 minutes and closely followed by *Frontier Airlines Inc.* with around 13 minutes.

The problem with the mean values are, that some delays are over a day long, which could influence the mean pretty heavily, if there are more than just a few. To get a look at this, the delays are split into groups, defined as less than 10 minutes as a short delay, 10 to 60 minutes as a medium delay and more than 60 minutes as a long delay. To get a good overview of how many flights are delayed in the groups, the next plot shows the distribution of the delays in percentage. By this you can see how many of all flights had a short,medium or long delay grouped by the airlines. One could argue, that the airlines with the least short delays has the more medium and long delays. Looking at that, the worst airline is *Spirit Air Lines* with roughly 70% short delays, ~21% medium delays and nearly 10% long delays, followed by *United Air Lines* with around 72% short delays, 20% and around 8% long delays. On the third place of the worst airlines is *Frontier Airlines Inc.* with ~74% short delays, nearly 18% medium delays and 8% long delays. By looking at these both factors the airlines with the most delays based on the mean and distribution the duration of the delay are:
1. Spirit Air Lines
2. United Air Liens
3. Frontier Air Lines Inc.

# 5. Predicting flight delays with ML


Since the EDA focused on delays, the main goal of the ML model to implement should be to predict a delay. In this case the model should be able to predict the flight delay. After getting 3 weeks of training data, the model they should be able to predict the delays of the last week. The decision was made to take the *United Air Lines Inc.* because it's the one with the most flights on record from the top 3 most delayed airlines. The month August was randomly chosen.

In [ ]:
flights_data_swa = flights_data[flights_data["airline_name"] == "Southwest Airlines Co."]
flights_data_to_predict = flights_data_swa.copy(deep=True)
flights_data_to_predict = flights_data_to_predict[flights_data_to_predict["month"] == 8]
flights_data_to_predict = flights_data_to_predict[flights_data_to_predict["cancelled"] == 0]
flights_data_to_predict = flights_data_to_predict.reset_index()
flights_data_to_predict

Drop unnecessary columns and columns with NaN values

In [ ]:
flights_data_to_predict = flights_data_to_predict.drop(columns = ["airline_code", "diverted", "cancelled",
                                                                 "cancellation_reason", "air_system_delay",
                                                                 "security_delay", "airline_delay", "late_aircraft_delay",
                                                                 "weather_delay", "airline_name", "arrival_delay", "index",
                                                                 "elapsed_time", "air_time", "taxi_in", "delay_difference"
                                                                 ])

In [ ]:
stat_predict = pd.DataFrame(flights_data_to_predict["departure_delay"].describe())
stat_predict

In [ ]:
flights_data_to_predict["departure_delay"].plot(kind="hist")

To get more accurate results, it could be wise to filter out outliers. First idea was to only use the delays that are less than 60 minutes.

In [ ]:
stat_predict = pd.DataFrame(flights_data_to_predict["departure_delay"][flights_data_to_predict["departure_delay"]<60].describe())
stat_predict

In [ ]:
flights_data_to_predict["departure_delay"][flights_data_to_predict["departure_delay"]<60].plot(kind="hist")

By filtering the delays to under 60 minutes, we lose around 10% but on the other hand have a better distribution of the delays and probably get better prediction results.

In [ ]:
flights_data_to_predict = flights_data_to_predict[flights_data_to_predict["departure_delay"]<60]
flights_data_to_predict

First we need to convert the strings into integer representations

In [ ]:
string_columns = ["tail_number","origin_airport","destination_airport"] # "airline_name"
for c in string_columns:
    flights_data_to_predict[c] = pd.factorize(flights_data_to_predict[c])[0]
flights_data_to_predict

Handle dates for regression

In [ ]:
def convert_to_ordinal(timestamp):
    return (timestamp.hour * 60 + timestamp.minute)*60 +timestamp.second

drop empty values and format the rest of the dates

In [ ]:
date_columns = ["scheduled_departure", "departure_time", "wheels_off", "wheels_on", "scheduled_arrival", "arrival_time"]
flights_data_to_predict = flights_data_to_predict[flights_data_to_predict["wheels_on"].notna()]

In [ ]:
for c in date_columns:
    print(type(flights_data_to_predict[c].iloc[0]))
    flights_data_to_predict[c] = flights_data_to_predict[c].apply(convert_to_ordinal)
flights_data_to_predict

To evaluate the model later on the dataset is split into a training- (first 3 weeks of august) and a testset (last week of august). 

In [ ]:
train = flights_data_to_predict[flights_data_to_predict["date"].apply(lambda x:x.date()) < datetime.date(2015, 8, 23)]
train = train.drop(columns=["date"])
test = flights_data_to_predict[flights_data_to_predict["date"].apply(lambda x:x.date()) >= datetime.date(2015, 8, 23)]
test = test.drop(columns=["date"])
test

One basic approach to predict values is to implement a Random Forest. In this case the sklearn RandomForestRegressor is used with 100 estimators. 

In [ ]:
train_y = train["departure_delay"]
train_X = train.drop(columns=["departure_delay"])
test_y = test["departure_delay"]
test_X = test.drop(columns=["departure_delay"])

In [ ]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf.fit(train_X, train_y)

In [ ]:
predictions = rf.predict(test_X)

Model assessment approach:
In this section, the evaluation approach of this project is described. For both prediction
cases, a model was trained based on prepared data sets. To evaluate the reliability of the
trained models, the r2_score or the accuracy_score and mean_squared_error functions
are used. The r2_score is suitable for regression model and returns a value between
0 and 100%. The accuracy score is a classification score and returns a value between 0 and 100%.
In both cases does a higher value represent a better model

In [ ]:
r2 = r2_score(test_y, predictions)
mse = mean_squared_error(test_y, predictions)
print("Accuracy for the model is",(r2*100).round(2) , "% with a Mean Squared Error of:", mse)
#0.9536189195181843 6.565784486314211

For comparison increase the estimators to 1000 and compare it with the first Random Forest with 100 estimators.

In [ ]:
rf_two = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf_two.fit(train_X, train_y)

In [ ]:
predictions_two = rf_two.predict(test_X)

In [ ]:
r2_two = r2_score(test_y, predictions_two)
mse_two = mean_squared_error(test_y, predictions_two)
print("Accuracy for the model is",(r2_two*100).round(2) , "% with a Mean Squared Error of:", mse_two)

# 6. Prediction of the flights destination

In [ ]:
df = airports_flight_data_na_filled_orig[airports_flight_data_na_filled_orig['month'] == 8].copy()
df.drop(inplace=True, columns=['year', 'day', 'latitude', 'longitude', 'city', 'origin_airport_cat', 'destination_airport_cat', 'origin_airport_name', 'airline_cat', 'tail_number', 'flight_number', 'cancelled', 'cancellation_reason','state', 'country', 'delay_difference', 'air_system_delay', 'security_delay', 'airline_delay', 'late_aircraft_delay', 'weather_delay', 'departure_time', 'arrival_time'])

In [ ]:
def convert_to_ordinal(timestamp):
    return (timestamp.hour * 60 + timestamp.minute)*60 +timestamp.second

In [ ]:
date_columns = ["scheduled_departure", "wheels_off", "wheels_on", "scheduled_arrival"]
df = df[df["wheels_on"].notna()]

for c in date_columns:
    df[c] = df[c].apply(convert_to_ordinal)
df

In [ ]:
airports_data = airports_data.reset_index()
df = df.merge(airports_data[['index', 'iata_code']], left_on='origin_airport', right_on='iata_code')
df.rename(columns={'index': 'origin_airport_encoded'}, inplace=True)
df.drop(inplace=True, columns='iata_code')
df = df.merge(airports_data[['index', 'iata_code']], left_on='destination_airport', right_on='iata_code')
df.rename(columns={'index': 'destination_airport_encoded'}, inplace=True)
df.drop(inplace=True, columns='iata_code')
df = df.dropna()

In [ ]:
df.head()

In [ ]:
top_50_airports_in_august = df.groupby(by='destination_airport').size().sort_values(ascending=False)[:50].index
top_50_airports_in_august

In [ ]:
df = df[df['origin_airport'].isin(top_50_airports_in_august)]
df = df[df['destination_airport'].isin(top_50_airports_in_august)]
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['scheduled_departure','departure_delay','taxi_out','wheels_off','scheduled_time','distance','wheels_on','taxi_in','scheduled_arrival','arrival_delay','diverted','origin_airport_encoded']], df['destination_airport_encoded'], test_size=0.2, shuffle=True)

In [ ]:
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
rf.fit(X_train, y_train)

Model assessment approach:
In this section, the evaluation approach of this project is described. For both prediction
cases, a model was trained based on prepared data sets. To evaluate the reliability of the
trained models, the r2_score or the accuracy_score and mean_squared_error functions
are used. The r2_score is suitable for regression model and returns a value between
0 and 100%. The accuracy score is a classification score and returns a value between 0 and 100%.
In both cases does a higher value represent a better model

In [ ]:
predictions = rf.predict(X_test)
acc = accuracy_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
print("Accuracy for the model is",(acc*100).round(2) , "% with a Mean Squared Error of:", mse)

For comparison increase the estimators to 1000 and compare it with the first Random Forest with 100 estimators.

In [ ]:
rf_two = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf_two.fit(X_train, y_train)

In [ ]:
predictions_two = rf_two.predict(X_test)
acc_two = accuracy_score(y_test, predictions_two)
mse_two = mean_squared_error(y_test, predictions_two)

print("Accuracy for the model is",(acc_two*100).round(2) , "% with a Mean Squared Error of:", mse_two)

# 7. Evaluation of the results
### Evaluation of the delay prediction
A regression model was developed to predict delays based on a given subset of data.
In the prepared evaluation setup, the r2_score and mean_squared_error were calculated.
For this particular model the r2_score determines an accuracy of  98.81% and an
MSE of around 1.417 which can be seen as a good value.

Even though the model is already very good, the same procedure was repeated with
    an n_estimator of 1000. This was done to test if the modelled could be improved by
increasing the amount of decision trees. This was a success, but with little changes. This
particular model achieved an improvement by 0.08% for the accuracy and the MSE was
reduced by 0.09. This improvement does not impact the prediction too much, but still
slightly enhanced the model, which concludes that a higher n_estimator can improve a
random forest, even though the model is already good.

In terms of the initial hypothesis it was shown that a flight delay can be predicted by its attributes.

### Evaluation of the flight destination prediction.
In order to predict the destination airport of a flight, a classification model was trained on a given subset of data.
The data was divided into training and test data subsets with the train_test_split by sklearn.

The evaluation scores for the destination airport prediction model shows an accuracy_score of 98,51% but at the same time an MSE score of 167,211. This seems like a high error value for the calculated accuracy. In total, it seems to be an accurate model.

As a comparison the random forest classifier should also train with an n_estimators of 1000.
The higher number of estimators results in an improvement in accuracy of 0,07%. At the same time the MSE was reduced by 8,031.

Due to the fact that the MSE is a metric value where the models need to be compared, a high value does not represent a bad model. A possible explanation could be that the airports are numerical encoded and a wrong classification is weighted differently based on the classified airport.

Regarding the hypothesis whether a flights destination can be predicted by its attributes, the result shows a good prediction score on the test data. This confirms the hypothesis.